In [1]:
import pandas as pd

In [2]:
import os

In [3]:
for file in os.listdir("data"):
    print(file)
    df = pd.read_csv("data/" + file)
    print(df.columns)

salaries.csv
Index(['emp_no', 'salary'], dtype='object')
dept_emp.csv
Index(['emp_no', 'dept_no'], dtype='object')
dept_manager.csv
Index(['dept_no', 'emp_no'], dtype='object')
departments.csv
Index(['dept_no', 'dept_name'], dtype='object')
titles.csv
Index(['title_id', 'title'], dtype='object')
employees.csv
Index(['emp_no', 'emp_title_id', 'birth_date', 'first_name', 'last_name',
       'sex', 'hire_date'],
      dtype='object')


In [4]:
import sqlite3
import pandas as pd

# Create a new SQLite database
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Define SQL for creating tables
create_tables_sql = {
    'departments': '''
        CREATE TABLE IF NOT EXISTS departments (
            dept_no VARCHAR PRIMARY KEY,
            dept_name VARCHAR NOT NULL
        );
    ''',
    'dept_emp': '''
        CREATE TABLE IF NOT EXISTS dept_emp (
            emp_no INT,
            dept_no VARCHAR,
            FOREIGN KEY (emp_no) REFERENCES employees(emp_no),
            FOREIGN KEY (dept_no) REFERENCES departments(dept_no)
        );
    ''',
    'dept_manager': '''
        CREATE TABLE IF NOT EXISTS dept_manager (
            dept_no VARCHAR,
            emp_no INT,
            FOREIGN KEY (dept_no) REFERENCES departments(dept_no),
            FOREIGN KEY (emp_no) REFERENCES employees(emp_no)
        );
    ''',
    'employees': '''
        CREATE TABLE IF NOT EXISTS employees (
            emp_no INT PRIMARY KEY,
            emp_title_id VARCHAR,
            birth_date DATE,
            first_name VARCHAR,
            last_name VARCHAR,
            sex VARCHAR,
            hire_date DATE,
            FOREIGN KEY (emp_title_id) REFERENCES titles(title_id)
        );
    ''',
    'salaries': '''
        CREATE TABLE IF NOT EXISTS salaries (
            emp_no INT,
            salary INT,
            FOREIGN KEY (emp_no) REFERENCES employees(emp_no)
        );
    ''',
    'titles': '''
        CREATE TABLE IF NOT EXISTS titles (
            title_id VARCHAR PRIMARY KEY,
            title VARCHAR
        );
    '''
}

# Create tables
for table_sql in create_tables_sql.values():
    cursor.execute(table_sql)

# Import data from CSV files into the tables
csv_files = {
    'departments': 'data/departments.csv',
    'dept_emp': 'data/dept_emp.csv',
    'dept_manager': 'data/dept_manager.csv',
    'employees': 'data/employees.csv',
    'salaries': 'data/salaries.csv',
    'titles': 'data/titles.csv',
}

for table_name, csv_file in csv_files.items():
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='append', index=False)

# Example query: List the employee number, last name, first name, sex, and salary of each employee
query = '''
SELECT e.emp_no, e.last_name, e.first_name, e.sex, s.salary
FROM employees e
JOIN salaries s ON e.emp_no = s.emp_no;
'''
df_query_result = pd.read_sql_query(query, conn)
print(df_query_result.head())

# Close the database connection
conn.close()


   emp_no last_name first_name sex  salary
0   10001   Facello     Georgi   M   60117
1   10002    Simmel    Bezalel   F   65828
2   10003   Bamford      Parto   M   40006
3   10004   Koblick  Chirstian   M   40054
4   10005  Maliniak    Kyoichi   M   78228


In [6]:

# Connect to the SQLite database
conn = sqlite3.connect('database.db')

# Data Analysis Queries
queries = {
    "list_employee_details": '''
        SELECT e.emp_no, e.last_name, e.first_name, e.sex, s.salary
        FROM employees e
        JOIN salaries s ON e.emp_no = s.emp_no;
    ''',
    "employees_hired_in_1986": '''
        SELECT first_name, last_name, hire_date
        FROM employees
        WHERE strftime('%Y', hire_date) = '1986';
    ''',
    "list_department_managers": '''
        SELECT d.dept_no, d.dept_name, dm.emp_no, e.last_name, e.first_name
        FROM dept_manager dm
        JOIN departments d ON dm.dept_no = d.dept_no
        JOIN employees e ON dm.emp_no = e.emp_no;
    ''',
    "list_department_employee_numbers": '''
        SELECT de.dept_no, e.emp_no, e.last_name, e.first_name, d.dept_name
        FROM dept_emp de
        JOIN employees e ON de.emp_no = e.emp_no
        JOIN departments d ON de.dept_no = d.dept_no;
    ''',
    "hercules_b": '''
        SELECT first_name, last_name, sex
        FROM employees
        WHERE first_name = 'Hercules' AND last_name LIKE 'B%';
    ''',
    "employees_in_sales": '''
        SELECT e.emp_no, e.last_name, e.first_name
        FROM employees e
        JOIN dept_emp de ON e.emp_no = de.emp_no
        JOIN departments d ON de.dept_no = d.dept_no
        WHERE d.dept_name = 'Sales';
    ''',
    "employees_in_sales_and_dev": '''
        SELECT e.emp_no, e.last_name, e.first_name, d.dept_name
        FROM employees e
        JOIN dept_emp de ON e.emp_no = de.emp_no
        JOIN departments d ON de.dept_no = d.dept_no
        WHERE d.dept_name IN ('Sales', 'Development');
    ''',
    "frequency_last_names": '''
        SELECT last_name, COUNT(*) as frequency
        FROM employees
        GROUP BY last_name
        ORDER BY frequency DESC;
    '''
}

# Execute each query and print the first few rows of the results
for query_name, query_sql in queries.items():
    print(f"Results for {query_name}:")
    df_query_result = pd.read_sql_query(query_sql, conn)
    print("Number of rows:", len(df_query_result))
    print(df_query_result.head(), "\n")

# Close the database connection
conn.close()


Results for list_employee_details:
Number of rows: 300024
   emp_no last_name first_name sex  salary
0   10001   Facello     Georgi   M   60117
1   10002    Simmel    Bezalel   F   65828
2   10003   Bamford      Parto   M   40006
3   10004   Koblick  Chirstian   M   40054
4   10005  Maliniak    Kyoichi   M   78228 

Results for employees_hired_in_1986:
Number of rows: 0
Empty DataFrame
Columns: [first_name, last_name, hire_date]
Index: [] 

Results for list_department_managers:
Number of rows: 24
  dept_no        dept_name  emp_no     last_name first_name
0    d001        Marketing  110022    Markovitch  Margareta
1    d001        Marketing  110039      Minakawa   Vishwani
2    d002          Finance  110085         Alpin       Ebru
3    d002          Finance  110114    Legleitner      Isamu
4    d003  Human Resources  110183  Ossenbruggen    Shirish 

Results for list_department_employee_numbers:
Number of rows: 331603
  dept_no  emp_no last_name first_name        dept_name
0    d005  